#### importer les libs

In [4]:
import pandas as pd
import sklearn as sk
from sklearn import model_selection
from sklearn import linear_model

#### déclarer les classes

In [5]:
class DataHandler:
    def __init__(self):
        self.price = None
        self.listing = None
        self.merge = None
        ;

    def get_data(self):
        self.price = pd.read_csv("https://storage.googleapis.com/h3-data/price_availability.csv", sep=";")
        self.listing = pd.read_csv("https://storage.googleapis.com/h3-data/listings_final.csv", sep=";")

    def group_data(self):
        self.listing.drop("Unnamed: 0", axis="columns", inplace=True)
        listing_price_mean = self.price.groupby("listing_id")['local_price'].mean()
        self.merge = self.listing.merge(listing_price_mean, on="listing_id")

    def get_process_data(self):
        self.get_data()
        self.group_data()
        return self.price, self.listing
        ;

#### commencer le processing

In [ ]:
d = DataHandler()
listing, prices = d.get_process_data()
